In [ ]:
# English algorithm version

In [ ]:
###### Module Loading #######

In [ ]:
Pandas, os, scipy and numpy modules are loaded.

In [ ]:
###### Move to working directory #######

In [ ]:
set working directory '/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/RNA-seq/'

In [ ]:
###### Load GTEX sample metadata and sample names #######

In [ ]:
load GTEX sample tissue attributes '/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt', columns='SAMPID','SMTS','SMTSD'

In [ ]:
load sample name list from '../GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct.gz'

In [ ]:
merge sample name and tissue attributes

In [ ]:
unique_tissues list from GTEX attributes

In [ ]:
###### Filter TPM file for transcripts with very low T test value for the different tissues #######
"""
GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct.gz is read chunk by chunk, for each line in the chunk
a t-test is performed for each unique tissue. Transcripts with 1 to 3 transcript-tissue t-test values of 0 are
forwarded to two new files:
1) Top_T-test_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct: Contains the raw TPM values as read.
2) Binary_top_T-test_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct: Contains the T-test values for 
    each tissue converted to 0 or 1 according to the values being under or over a threshold (0.0001 by default).
"""

In [ ]:
#cols=samples of the same tissue type, notcols = samples of different tissue type
"cols" and "notcols" = empty dictionary
for each tissue, populate "cols" and "notcols" dictionaries
"count" and "added" = 0, "chunksize" = 1000, and "write_header" = True

In [ ]:
read '../GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct.gz' in chunks of "chunksize", dropping 'gene_id' column
for each chunk:
    "bin_output" and "output" = empty df
    for each transcript in chunk:
        "ttest" = empty list
        for each "unique_tissues"
            append T-test pval for the tissue-transcript vs not_tissue-transcript to "ttest" list
            if more than 3 tissues have DE, move to the next
        if "ttest" has between 0 and 3 DE tissues:
            add transcript to "output"
            convert pvalues under 0.0001 and above 0 (WHY 0?) to 1, otherwise to 0 and add to "bin_output"
            
    if there is any "output": 
        export "output" 'Top_T-test_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct'
        and "bin_output" to 'Binary_top_T-test_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct'
    set write_header to False #Avoid exporting the header every chunk
    "count"="count" + "chunksize"
    "added"="added" + length of "output"
    print summary

In [ ]:
###### Load previous 2) output #######

In [ ]:
bin_ttest = read 'Binary_top_T-test_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct'

In [ ]:
###### Transcript selection #######
"""
100 transcripts are selected for each tissue, an iterative selection is performed increasing the threshold
(number of tissue with significant T-test value for the transcript). Transcripts with fewer total associations
are selected first to maximise the tissue information each transcript can provide.
"""

In [ ]:
"data_selection" = empty dict
"threshold" = 1
for "tissue" in "bin_ttest" columns:
    while "tissue" not in "data_selection":
        "data" = "bin_ttest" where sum of values between 0 and "threshold"
        if length of "data" >= 100:
            "data_selection[tissue]" = first 100 transcripts
        if "threshold" == number of tissues: break
        "threshold" = "threshold" + 1

In [ ]:
"index_selection" = set of transcripts in "data_selection" 

In [ ]:
###### Read raw data for selected transcripts #######
"""
The complete list of transcripts is read from the previously exported 1) file, then index of the transcripts
matching the selection is used to read only the raw values of the selected transcripts.
"""

In [ ]:
"transcripts" = read 'Top_T-test_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct'


In [ ]:
"rows" = "transcripts" index if in "index_selection"

In [ ]:
data = read 'Top_T-test_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct' using "rows" index


In [ ]:
###### HKG raw data #######
"""
REVIEW:
Raw data for a published list of Housekeeping Genes is extracted from GTEX data. Transcripts with at least 1 TPM
are filtered and the top 150 with less relative variation are selected.
Publication:
Wang, Z., Lyu, Z., Pan, L. et al. Defining housekeeping genes suitable for RNA-seq analysis of the human allograft kidney biopsy tissue. BMC Med Genomics 12, 86 (2019). https://doi.org/10.1186/s12920-019-0538-z
"""

In [ ]:
#Dictionary of Housekeeping genes and ID
"HKG" = {'ANAPC5':'ENSG00000089053','ANAPC15':'ENSG00000110200','ARID3B':'ENSG00000179361','ARL10':'ENSG00000175414','ATXN2':'ENSG00000204842','C3orf49':'ENSG00000163632','CCAR1':'ENSG00000060339','CCDC125':'ENSG00000277868','CCDC90B':'ENSG00000137500','CHFR':'ENSG00000072609','DHRSX':'ENSG00000169084','FRMD8':'ENSG00000126391','GGA1':'ENSG00000100083','HERC4':'ENSG00000148634','MKNK1':'ENSG00000079277','NASP':'ENSG00000132780','NME4':'ENSG00000103202','OTUB1':'ENSG00000167770','POLR2B':'ENSG00000047315','POLR3A':'ENSG00000148606','POMK':'ENSG00000185900','PSMA3-AS1':'ENSG00000257621','PTPN14':'ENSG00000152104','RAPGEF6':'ENSG00000158987','REL':'ENSG00000162924','RRP1':'ENSG00000160214','RUNDC1':'ENSG00000198863','SAMD4B':'ENSG00000179134','SLC4A1AP':'ENSG00000163798','SLMAP':'ENSG00000163681','SMARCAL1':'ENSG00000138375','SNAP29':'ENSG00000099940','SNRNP200':'ENSG00000144028','SUPT4H1':'ENSG00000213246','TBC1D22A':'ENSG00000054611','THUMPD3-AS1':'ENSG00000206573','TSPOAP1-AS1':'ENSG00000265148','TUBGCP2':'ENSG00000130640','WDTC1':'ENSG00000142784','ZNF544':'ENSG00000198131'}

In [ ]:
"count"=0
"added"=0
"write_header"=True

In [ ]:
for chunk in read '../GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct.gz'
    "output"=transcripts in chunk if gene_id in "HKG"
    if "output" not empty: 
        "output" to 'HKG_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct'
        "write_header"=False
    "count"="count"+"chunksize"
    "added"="added"+length "output"
    print summary

In [ ]:
"HKG_raw" = read 'HKG_GTEx_Analysis_2017-06-05_v8_RSEMv1.3.0_transcript_tpm.gct'

In [ ]:
"HKG_top" = "HKG_raw" where mean TPM > 1

In [ ]:
"HKG_top_rel_data" = "HKG_top" / "HKG_top" mean

In [ ]:
"HKG_transc" = "HKG_top_rel_data" 150 HKG transcripts with smallest STDEV

In [ ]:
###### Normalize GTEX data #######
"""
Raw data for transcripts selected before are normalized by dividing raw TPM by the mean expression 
of the Housekeeping Genes filtered above for each sample.
"""

In [ ]:
"rel_data" = "data" / mean "HKG_raw" where transcripts = "HKG_transc"

In [ ]:
"data" = merge "samples" with "rel_data"

In [ ]:
"data" to 'Normalised_data_for_prediction.csv.gz'

In [ ]:
data = read 'Normalised_data_for_prediction.csv.gz' # In case of notebook killed, data can be restored

In [ ]:
"reindex_data" = "data" with new index

In [ ]:
###### Identify significant (DE) transcripts for each tissue #######
"""
A T-test is used to assess the p-value of the expression of every transcript of being differentially expressed
for every tissue. If the p-value falls under a certain threshold, the transcripts is considered to have a DE
for such tissue. This section is currently not necessary to run the whole script!
"""

In [ ]:
"significant" = {}

In [ ]:
for "Tissue" in data.index.unique(): significant[Tissue]=[]

In [ ]:
for transcripts in "data":
    for Tissue in "data":
        "ttest" = ttest_ind (scipy) of "data[Transc][Tissue]" against "data[Transc][data != Tissue]"
        if "ttest" pvalue <= 0.00001:
            append transcripts to "significant[Tissue]"

In [ ]:
####### Calculate ratio of success for each transcript for each tissue #######
"""
For each tissue type a T-test is used to assess the p-value of that transcript of being differentially 
expressed in that tissue. If so, for each sample of the same tissue a two-sided t-test is used to obtain
the p-value of the sample expression of belonging to a normal distribution of the whole tissue-transcript
expression. If the p-value falls under a certain threshold, that sample is considered to be successfully
associated to the rest of samples of the same tissue, otherwise if the p-value is above the threshold
that sample is considered to fail to associate to the rest of samples of the same tissue. After testing 
all samples, sucesses and fails are used as a score of how likely a DE expressed transcript would have a
significant expression in one sample of a specific tissue.
"""

In [ ]:
"fails"={}
"success"={}
"ratio"={}

In [ ]:
"len_tran"=length "data" columns #Number of transcripts in data
for Tissue in "data":
    print Tissue
    "fails[Tissue]"=[]
    "success[Tissue]"=[]
    "ratio[Tissue]"={}
    for Transcript in "data":
        "ttest_tissue" = ttest_ind (scipy) of "data[Transc][Tissue]" against "data[Transc][data != Tissue]"
        if "ttest_tissue" pvalue <= 0.00001:
            for sample in data:
                "ttest_sample" = stats.ttest_1samp (scipy) of "data[Tissue][Transcript]" against "data.iloc[sample][Transcript]"
                if "ttest_sample" pvalue <= 0.0001: append Transcript to "success[Tissue]"
                elif "ttest_sample" pvalue >= 0.0001: append Transcript to "fails[Tissue]"
                if "data[Transc][Tissue]" expression == 0 and "sample[Transc]" expression == 0: append Transcript to "success[Tissue]" #T-test can't be performed if expression = 0
                if "data[Transc][Tissue]" expression == 0 and "sample[Transc]" expression != 0: append Transcript to "fails[Tissue]" #T-test can't be performed if expression = 0
            "ratio[Tissue][Transc]" = ratio of transcript success / failure for Tissue
        else: "ratio[Tissue][Transc]" = NaN # If p-value above 0.00001, no ratio
    print summary

In [ ]:
"ratiodata" = DataFrame of "ratio"

In [ ]:
"ratiodata" to 'Ratiodata_v8.csv'

In [ ]:
"ratiodata" = read 'Ratiodata_v8.csv'

In [ ]:
# Only transcripts with a high score are used.
for every data in "ratiodata", if <=0.7: turn into NaN

In [ ]:
# To avoid transcripts associated to multiple tissues being over-informative, a relative score of each 
# transcript-tissue association can be used insted.
"rel_ratiodata"= "ratiodata" / "ratiodata" sum

In [ ]:
####### Transcript-tissue scoring function ######
"""
A simple sample relative expression set is passed to this function. For each transcript selected previously, 
a T-test is performed to obtain the p-value of the expression of such transcript in the sample to belong to 
the expression pattern of each tissue. If the p-value falls under a certain threshold, the combination 
transcript-tissue gets a scoring equal to the ratio of success of such transcript for that tissue as calculated 
above. The list of transcript-tissue scores is returned to calculate the weights.
"""

In [ ]:
"t_test_ratio_external" function given sample "i":
    "value"={}
    for Transcript in "data":
        "value[Transc]"={}
        for Tissue in "data":
            "ttest_sample"= stats.ttest_1samp (scipy) of "data[Transc][Tissue]" against "i[Transc]"
            "value[Transc][Tissue]"=NaN
            if "ttest_sample" pvalue >= 0.0001: "value[Transc][Tissue]" = "ratiodata[Transc][Tissue]"
    return "value"

In [ ]:
##### LOAD eQTLs #######
"""
Pre-finemapped eQTLs from GTEX database are loaded to make predictions based on attributed weights for each tissue.
Only CaVEMaN dataset is being used here although a consensus dataset is recommended.
"""

In [ ]:
"eQTLs" read 'GTEx_v8_finemapping_CaVEMaN.txt.gz'

In [ ]:
"eQTLs" = "eQTLs" pivot table with index=['eQTL','GENE'] columns='TISSUE' and values='Probability'

In [ ]:
##### Tissue name link #######
"""
Since tissue names differ, specially in format, in eQTL and expression data, a dictionary of corresponding names
is used to match both names when scoring.
"""

In [ ]:
"tissues_common"={}
for Tissue in "data":
    if Tissue in "eQTLs":
        "tissues_common[b]"= Tissue
    else:
        if Tissue with str replacements in "eQTLs":
            "tissues_common[Tissue]" = Tissue with str replacements
        else:
            print warning

In [ ]:
"tissues_common_inv" = reverse dictionary of "tissues_common"

In [ ]:
###### Prediction on the mean RNA-seq value per tissue #######
"""
To test the scoring system, optimal-conditions samples are obtain with the mean transcript expression for each
tissue. This mean-sample should align much better to eQTL values than single samples randomly obtain from the dataset.
"""

In [ ]:
"tissue_means"= "data" mean grouped by Tissue

In [ ]:
####### Prediction and eQTL comparison for each mean RNA-seq value ######
"""
Each mean-expression sample is used as a target for weighted means prediction and compared to simple mean
prediction and to real eQTL value for that tissue.
"""

In [ ]:
"prediction"={}
"real"={}
"mean_prediction"={}
### a = renamed eQTLs to match RNAseq names
"a" =  "eQTLs" with renamed Tissue names
for Tissue in "tissues_common":
    "test"=t_test_ratio_external (function) of mean-expression sample of Tissue
    "b" = tissue scores from "test" 
    "b_where_a" = scores from "b" where "eQTLs" has value for such transcript
    "b_where_a_sum" = "b_where_a" sum (sum of scores)
    "prediction[Tissue]" = sum of ("eQTLs" multiplied by ("b_where_a" divided by "b_where_a_sum")) # sum of eQTL values multiplied by scores divided by sum of scores
    "real[Tissue]" = "eQTLs" value for Tissue
    "mean_prediction[Tissue]" = "eQTLs" mean

In [ ]:
"Melted_results"= concatenated Real, Mean and Predicted results and melted into single transcript-SNP values

In [ ]:
"Melted_results" to 'Results_to_plot_2307.csv'

In [ ]:
###### Perform predictions on random RNA-seq samples and check performance per tissue #######

In [ ]:
"a" = renamed "eQTLs"
"mean_ratio"=[]
"pred_ratio"=[]
"slope"=[]
"weighted_predicted"=[]
"mean_predicted"=[]
"tissue"=[]
"count"=0
while "count" < 1000:
    for sample in "data":
        if sample tissue in "tissues_common":
            if "eQTLs" value for tissue and sample:
                "test" = t_test_ratio_external (function) of sample
                "b" = tissue scores from "test" 
                "b_where_a" = scores from "b" where "eQTLs" has value for such transcript
                "b_where_a_sum" = "b_where_a" sum (sum of scores)
                "prediction" = "eQTLs" multiplied by ("b_where_a" divided by "b_where_a_sum") # eQTL values multiplied by scores divided by sum of scores
                "real" = "eQTLs" value for Tissue
                "mean_prediction" = "eQTLs" mean
                if "mean_prediction" is NaN: skip 
                print ratios
                "mean_ratio" append "real"/"mean_prediction"
                "pred_ratio" append "real"/"prediction"
                "slope" append "real"
                "weighted_predicted" append "prediction"
                "mean_predicted" append "mean_prediction"
                "tissue" append sample tissue
                "count"="count"+1

In [ ]:
Results_means.merge(pd.DataFrame(ratiodata.count(axis=1),columns=['Sig Transc']),how='left',left_index=True,right_index=True).merge(pd.DataFrame((ratiodata/ratiodata.sum()).mean(axis=1),columns=['Mean sig transc score']),how='left',left_index=True,right_index=True).to_csv('Results_to_plot_2307.csv',index=True)